This is a sample of a RAG application using both Langchain and Ollama, that reads a pdf or text file.

In [3]:
from langchain_community.llms import Ollama
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import OllamaEmbeddings

In [4]:
# Change the model parameter for prefered llm
llm = Ollama(model="openhermes")

In [5]:
# Load text file
#loader = TextLoader("./the_great_gatsby.txt")
#doc = loader.load()

In [6]:
# Load pdf document
loader = PyPDFLoader("document path")
doc = loader.load_and_split()

In [7]:
# Spliting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(doc)

# Index the text chunks
embbed = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embbed)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [8]:
# Use default prompt from langchain for rag apps
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [9]:
# Format the response
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
for chunk in rag_chain.stream("What is the pdf about?"):
    print(chunk, end="", flush=True)

The pdf is about a method called StoryDiffusion for generating subject-consistent images and transition videos using storytelling prompts. It introduces a Semantic Motion Predictor for video generation, focuses on continuous transitions between images, and demonstrates strong performance in generating consistent and seamless transition videos. The method also explores the integration of Consistent Self-Attention with ControlNet for pose control in image generation and has plug-and-play capability.

In [11]:
# cleanup the vectorstore
#vectorstore.delete_collection()